# CNN using Tensorflow Keras on MRI Image Data

## Data Use Agreements
The data used for this project were provided in part by OASIS and ADNI.

OASIS-3: Principal Investigators: T. Benzinger, D. Marcus, J. Morris; NIH P50 AG00561, P30 NS09857781, P01 AG026276, P01 AG003991, R01 AG043434, UL1 TR000448, R01 EB009352. AV-45 doses were provided by Avid Radiopharmaceuticals, a wholly owned subsidiary of Eli Lilly.

Data collection for this project was done through the Alzheimer's Disease Neuroimaging Initiative (ADNI) (National Institutes of Health Grant U01 AG024904) and DOD ADNI (Department of Defense award number W81XWH-12-2-0012). ADNI is funded by the National Institute on Aging, the National Institute of Biomedical Imaging and Bioengineering, and through generous contributions from the following: AbbVie, Alzheimer’s Association; Alzheimer’s Drug Discovery Foundation; Araclon Biotech; BioClinica, Inc.; Biogen; Bristol-Myers Squibb Company; CereSpir, Inc.; Cogstate; Eisai Inc.; Elan Pharmaceuticals, Inc.; Eli Lilly and Company; EuroImmun; F. Hoffmann-La Roche Ltd and its affiliated company Genentech, Inc.; Fujirebio; GE Healthcare; IXICO Ltd.; Janssen Alzheimer Immunotherapy Research & Development, LLC.; Johnson & Johnson Pharmaceutical Research & Development LLC.; Lumosity; Lundbeck; Merck & Co., Inc.; Meso Scale Diagnostics, LLC.; NeuroRx Research; Neurotrack Technologies; Novartis Pharmaceuticals Corporation; Pfizer Inc.; Piramal Imaging; Servier; Takeda Pharmaceutical Company; and Transition Therapeutics. The Canadian Institutes of Health Research is providing funds to support ADNI clinical sites in Canada. Private sector contributions are facilitated by the Foundation for the National Institutes of Health (www.fnih.org). The grantee organization is the Northern California Institute for Research and Education, and the study is coordinated by the Alzheimer’s Therapeutic Research Institute at the University of Southern California. ADNI data are disseminated by the Laboratory for Neuro Imaging at the University of Southern California.

## Setup

### Imports

In [4]:
import nibabel.freesurfer.mghformat as mgh

from tqdm.notebook import tqdm

import os, sys, shutil

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Set up and test Tensorflow

In [5]:
import tensorflow as tf
from tensorflow.keras import layers

In [6]:
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print()
device_name = tf.test.gpu_device_name()
if not device_name:
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Tensor Flow Version: 2.5.0
Keras Version: 2.5.0

Python 3.8.10 (default, Jun  4 2021, 15:09:15) 
[GCC 7.5.0]
Pandas 1.3.0

Found GPU at: /device:GPU:0


## Load data

### MRI scans
Since all the files are already transformed via the freesurfer, I don't think we'll need to do any major preprocessing like cropping, flipping, or rotating.
```
main_directory/
    control/
        mr_id_001/
            brain_image_001.mgz
            brain_image_001_transformed.mgz
            talairach_001.xfm
        mr_id_002/
            brain_image_002.mgz
            brain_image_002_transformed.mgz
            talairach_002.xfm
    dementia/
        mr_id_003/
            brain_image_003.mgz
            brain_image_003_transformed.mgz
            talairach_003.xfm
        mr_id_004/
            brain_image_004.mgz
            brain_image_004_transformed.mgz
            talairach_004.xfm
```

#### Define some flags for general use

In [ ]:
# This class allows you to access dictionary items with a dot
# Gathered from here: https://stackoverflow.com/questions/2352181/how-to-use-a-dot-to-access-members-of-dictionary
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [ ]:
FLAGS = {
    'rescale_fac' : .5,
    'scan_width'  : None,
    'scan_height' : None,
    'scan_depth'  : None,
    # 'data_dir'    : '/content/drive/MyDrive/Extracurriculars/Summer_2021/Polygence/Data/OASIS/mri_data',  # use this if mounting on drive
    'data_dir'    : '/home/jack/Code/GitHub/Polygence/Data/OASIS/mri_data',  # use this if on jupyter
    'batch_size'  : 8,
    'random_state': 47,
    'epochs'      : 1
}
FLAGS = dotdict(FLAGS)
print(FLAGS)

#### Generate the filenames
The returned filenames list will be organized as follows:
```
[[path_to_scan_1, label_for_scan_1],
 [path_to_scan_2, label_for_scan_2],
 [path_to_scan_3, label_for_scan_3],
 ...
 [path_to_scan_n, label_for_scan_n]]

```

In [ ]:
def generate_filenames(labels=['control', 'dementia']):
    pairs = []
    
    for label in labels:
        label_dir = os.path.join(FLAGS.data_dir, label)
        mr_ids = os.listdir(label_dir)
        mr_ids.sort()
        
        for mr_id in tqdm(mr_ids, desc=label):
            scans = os.path.join(label_dir, mr_id)
            img_file = [file for file in os.listdir(scans) if "transformed" in file]
            img_path = os.path.join(scans, img_file[0])
            i = 1 if label == 'dementia' else 0
            pairs.append([img_path, i])
    
    m = len(pairs)
    filenames = []
    labels = np.zeros((m, 1), dtype='int32')
    
    idx = 0
    for filename, label in pairs:
        filenames.append(filename)
        labels[idx, 0] = label
        idx += 1
    
    filenames = np.array(filenames)
    return filenames, labels
        

X_filenames, y_labels = generate_filenames()

### Split into training and validation set

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train_filenames, X_val_filenames, y_train, y_val = train_test_split(
    X_filenames, y_labels, test_size=.2, random_state=FLAGS.random_state, shuffle=True)

In [ ]:
print(X_train_filenames.shape, y_train.shape)
print(X_val_filenames.shape, y_val.shape)

In [ ]:
print(X_train_filenames[:5])
print(y_train[:5])

In [ ]:
print(X_val_filenames[:5])
print(y_val[:5])

#### Check to see if its a relatively even split between classes between train and test sets

In [ ]:
(unique, counts) = np.unique(y_labels, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)
print(f'{frequencies[0,1]/(np.sum(frequencies[:,1]))*100:.2f}% of the data is control')

In [ ]:
(unique, counts) = np.unique(y_train, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)
print(f'{frequencies[0,1]/(np.sum(frequencies[:,1]))*100:.2f}% of the data is control')

In [ ]:
(unique, counts) = np.unique(y_val, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)
print(f'{frequencies[0,1]/(np.sum(frequencies[:,1]))*100:.2f}% of the data is control')

### Create a custom generator
Since the data is too large to fit it all into memory, we will have to read it in batches.

In [ ]:
from skimage.transform import rescale

In [ ]:
class MRI_Data_Generator(tf.keras.utils.Sequence):
    """
    A data generator that reads MRI data in batches, and returns their image data
    """
    
    def __init__(self, filenames, labels, batch_size):
        """
        Intializes the generator
        :param filenames: list containing the path to each MRI scan file, should be np.ndarray
        :param labels: labels associated with the scans in filenames (control, dementia), should be np.ndarray
        :param batch_size: the size of the batch
        """
        self.filenames = filenames
        self.labels = labels
        self.batch_size = batch_size
        
    def __len__(self):
        """
        Calculate the number of batches that we are supposed to produce.
        Returns a rounded-up integer of total number of filenames divided by batch size.
        """
        return (np.ceil(len(self.filenames) / float(self.batch_size))).astype(np.int)
        
    def __getitem__(self, idx):
        """
        Scan the data within that batch
        :param idx: the index of the batch to be selected
        """
        # Read in the items at that batch index
        # Since these two arrays are np arrays, we don't have to worry about index_out_of_bounds
        batch_X = self.filenames[idx*self.batch_size : (idx+1)*self.batch_size]
        batch_y = self.labels[idx*self.batch_size : (idx+1)*self.batch_size]
        
        # Data preprocessing
        def normalize(volume):
            """ Normalize the volume, scaling it to [0, 1] instead of [0, 255] """
            min = 0.0
            max = 255.0
            volume[volume < min] = min
            volume[volume > max] = max
            volume = (volume - min) / (max - min)
            volume = volume.astype("float32")
            return volume
        
        shapes = set()

        def scale(volume):
            """ Reduce the volume's dimensions and crop"""
            volume = rescale(volume, FLAGS.rescale_fac)
            width, height, depth = volume.shape
            volume = volume[int(30/256 * width):int(220/256 * width), 
                        int(30/256 * height):int(220/256 * height), 
                        int(0/256 * depth):int(220/256 * depth)]
            
            FLAGS.scan_width, FLAGS.scan_height, FLAGS.scan_depth = volume.shape

            shapes.add(volume.shape)
            
            return volume
        
        batch_X_data = []
        for filename in batch_X:
            MRI_orig = mgh.load(filename)
            volume = MRI_orig.get_fdata()
            volume = normalize(volume)
            volume = scale(volume)
            batch_X_data.append(volume)
        
        np_res_X = np.array(batch_X_data)
        np_res_y = np.array(batch_y)
        
        if len(shapes) > 1:
            raise ValueError(f'There exists a non-standard shape: {shapes}')
        
        return np_res_X, np_res_y

### Test our data generator

In [ ]:
test_gen = MRI_Data_Generator(X_train_filenames, y_train, 4)

In [ ]:
batchx, batchy = test_gen.__getitem__(0)
print(">> Num of batches >> ", test_gen.__len__())  # ouputs 52, which is ceil(205 / 4), where 205 is size of validation set and 4 is batch size

In [ ]:
print(FLAGS)

In [ ]:
print(batchx.shape, batchy.shape)
print(batchy)

#### Plot the generated test results

In [ ]:
# Parameters for graph
nrows = 3
ncols = 4

# Set up figure and plot it
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

for i in range(4):
    sp = plt.subplot(nrows, ncols, i + 1)
#     sp.axis('Off') # Don't show axes (or gridlines)

    img = batchx[i][FLAGS.scan_width//2,:,:]
    plt.imshow(img)
    plt.title(f'control_{i+1}' if batchy[i] == 0 else f'dementia_{i+1}')
    
    sp = plt.subplot(nrows, ncols, i + 1 + 4)
#     sp.axis('Off') # Don't show axes (or gridlines)

    img = batchx[i][:,FLAGS.scan_height//2,:]
    plt.imshow(img)
    plt.title(f'control_{i+1}' if batchy[i] == 0 else f'dementia_{i+1}')
    
    sp = plt.subplot(nrows, ncols, i + 1 + 4 + 4)
    img = batchx[i][:,:,FLAGS.scan_depth//2]
    plt.imshow(img)
    plt.title(f'control_{i+1}' if batchy[i] == 0 else f'dementia_{i+1}')

In [ ]:
print(batchx[0].shape)

In [ ]:
print(FLAGS.scan_width, FLAGS.scan_height, FLAGS.scan_depth)

In [ ]:
# delete them to save memory
del batchx, batchy, test_gen

In [ ]:
training_batch_generator = MRI_Data_Generator(X_train_filenames, y_train, FLAGS.batch_size)
validation_batch_generator = MRI_Data_Generator(X_val_filenames, y_val, FLAGS.batch_size)

## CNN

#### Useful metrics
https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#understanding_useful_metrics

* False negatives and false positives are samples that were incorrectly classified
* True negatives and true positives are samples that were correctly classified
* Accuracy is the percentage of examples correctly classified
* Precision is the percentage of predicted positives that were correctly classified
* Recall is the percentage of actual positives that were correctly classified
* AUC refers to the Area Under the Curve of a Receiver Operating Characteristic curve (ROC-* AUC). This metric is equal to the probability that a classifier will rank a random positive sample higher than a random negative sample.
* AUPRC refers to Area Under the Curve of the Precision-Recall Curve. This metric computes precision-recall pairs for different probability thresholds.

In [ ]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=METRICS)

### Set up useful callbacks/things for training model

#### Tensorboard

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
!rm -rf ./logs/

In [ ]:
from datetime import datetime
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

In [ ]:
%tensorboard --logdir=logs --port=8008

#### Set up callbacks and learning rate scheduler to avoid overfitting/underfitting

In [ ]:
!rm -rf ./checkpoints/
filepath="checkpoints/model-epoch-{epoch:02d}-accuracy-{val_accuracy:.4f}-loss-{val_loss:.4f}.hdf5"
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1)

#### Create class weights, since there are less dementia samples than there are control

In [ ]:
(unique, counts) = np.unique(y_labels, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)
print(f'{frequencies[0,1]/(np.sum(frequencies[:,1]))*100:.2f}% of the data is control')

In [ ]:
# 0 is control, 1 is dementia
control_cnt = frequencies[0,1]
dementia_cnt = frequencies[1,1]
total_cnt = control_cnt + dementia_cnt
print(control_cnt, dementia_cnt, total_cnt)

In [ ]:
weight_for_0 = (1 / control_cnt) * (total_cnt / 2.0)
weight_for_1 = (1 / dementia_cnt) * (total_cnt / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}
print(class_weight)

In [ ]:
history = model.fit(
            training_batch_generator,
            steps_per_epoch = int(X_train_filenames.shape[0] // FLAGS.batch_size),  # samples = batch_size * steps
            epochs = FLAGS.epochs,
            verbose = 1,
            validation_data = validation_batch_generator,
            validation_steps = int(X_val_filenames.shape[0] // FLAGS.batch_size),  # samples = batch_size * steps
#             callbacks = [tensorboard_cb, checkpoint_cb, early_stopping_cb, lr_scheduler],
            callbacks = [tensorboard_cb, checkpoint_cb, lr_scheduler],
            class_weight = class_weight,
            use_multiprocessing = True,
            workers = 12,
            max_queue_size = 48)

### Evaluate the model

#### Plot metrics

In [ ]:
def plot_metrics(history):
    
    fig = plt.gcf()
    fig.set_size_inches(20, 3 * 4)

    metrics = ['auc','accuracy','loss', 'prc', 'precision', 'recall']
    for n, metric in enumerate(metrics):
        lims = {'loss': {0,2}}
        name = metric.replace("_"," ").capitalize()
        sp = plt.subplot(2, 3, n + 1)
        plt.plot(history.epoch, history.history[metric], color='C0', label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
                 color='C1', linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        if metric in lims:
            plt.ylim(lims[metric])

        plt.legend()

In [ ]:
plot_metrics(history)

#### Save model

In [ ]:
# Clear any models from previous runs
!rm -rf ./saved_model/
!mkdir -p saved_model
model.save('saved_model/my_model')

#### Confusion Matrix

In [ ]:
raw_predictions = model.predict(
    validation_batch_generator, verbose=1, max_queue_size=48,
    workers=12, use_multiprocessing=True
)

In [ ]:
cm = tf.math.confusion_matrix(labels=y_val.T[0], predictions=raw_predictions.T[0])

In [ ]:
sns.heatmap(cm, square=True, annot=True, fmt='d', cbar=False, cmap='Blues',
            xticklabels=['Control', 'Dementia'],
            yticklabels=['Control', 'Dementia'])
plt.xlabel('predicted label')
plt.ylabel('true label')
plt.title('Confusion matrix for validation set')